# Framework for predictions and portfolio forming

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time

# import the parquet library
import pyarrow.parquet as pq

# import model libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
# load 'basemodel.parquet'
#df = pd.read_parquet('basemodel.parquet')
df= pd.read_parquet('/kaggle/input/sign-prediction-datasets/clean_financials.parquet')
#df= pd.read_parquet('clean_financials.parquet')
prediction_cols = []
df.head()

,PERMNO,date,y,RET,LME,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,SPREAD_L,LTURNOVER,IVOL,IVOL2,IVOL3,STR,LTR,IMOM,MOM,PCTHIGH,MVOL,MVOL2,MVOL3,LMKT,LMKT2,LMKT3,MMOM,MIMOM,MLTR,release_L,LAT,DPI2A_L,CTO_L,D2A_L,NOA_L,OL_L,PCM_L,FC2Y_L,INVEST_L,RNA_L,S2E_L,PROF_L,PM_L,ATO_L,ROA_L,FCF_L,A2ME_L,B2ME_L,S2P_L,ROE_L,LEV_L,TQ_L
0,10000,1987-03-31,0,-0.384615,1581.53125,973.25000,0.0,0.0,0.0,1.0,0,0,0.076923,0.100694,0.000612,0.000830,0.002413,0.000000,-0.297252,-1.089044,-2.079441,0.091549,0.545027,0.771919,0.434190,0.0482,0.1289,-0.0278,0.186212,0.112764,0.459110,0.0,2.115,0.000000,0.055319,0.037352,0.000000,0.236407,-1.495726,1.777778,0.000000,0.016564,0.279904,0.064593,0.230769,0.071779,-0.061939,-0.024586,0.001067,0.000211,0.000059,-0.000083,0.000037,748.571277
1,10000,1987-04-30,0,-0.062500,973.25000,912.44134,0.0,0.0,0.0,0.0,0,1,0.625000,0.285384,0.003465,0.000612,0.000830,-0.384615,0.014185,-1.459321,-2.390877,0.062500,0.672597,0.545027,0.771919,0.0211,0.0482,0.1289,0.179935,-0.025601,0.498953,0.0,2.115,0.000000,0.055319,0.037352,0.000000,0.236407,-1.495726,1.777778,0.000000,0.016564,0.279904,0.064593,0.230769,0.071779,-0.061939,-0.024586,0.001067,0.000211,0.000059,-0.000135,0.000060,460.967849
2,10000,1987-05-31,0,-0.066667,912.44134,851.59375,0.0,0.0,0.0,0.0,0,2,0.133330,0.256358,0.001893,0.003465,0.000612,-0.062500,-0.089613,-1.633155,-2.772587,0.075378,1.565461,0.672597,0.545027,-0.0167,0.0211,0.0482,0.208747,0.032263,0.488026,1.0,2.115,-0.037825,0.074232,0.028309,-0.382033,0.293803,-1.101911,0.343949,-38.203310,-0.368640,-1.121429,2.535714,-2.261146,0.163032,-0.108274,-0.146901,0.000660,-0.000071,0.000079,-0.000251,0.000018,699.225968
3,10000,1987-06-30,0,0.000000,851.59375,851.59375,0.0,0.0,0.0,0.0,0,0,0.071451,0.209350,0.000190,0.001893,0.003465,-0.066667,-0.341485,-1.323014,-2.585254,0.070707,0.955605,1.565461,0.672597,0.0049,-0.0167,0.0211,0.142069,-0.002095,0.591159,0.0,2.115,-0.037825,0.074232,0.028309,-0.382033,0.293803,-1.101911,0.343949,-38.203310,-0.368640,-1.121429,2.535714,-2.261146,0.163032,-0.108274,-0.146901,0.000660,-0.000071,0.000079,-0.000269,0.000019,652.670811
4,10005,1987-03-31,0,0.000000,795.11688,795.11688,0.0,0.0,0.0,0.0,0,0,0.090936,0.016213,0.000532,0.021169,0.000013,0.100000,-0.405465,-1.163152,-0.470004,0.578954,0.545027,0.771919,0.434190,0.0482,0.1289,-0.0278,0.186212,0.112764,0.459110,0.0,1.954,-0.001485,0.023762,0.015865,-0.032673,0.044012,0.645833,1.437500,-3.267327,-0.423313,0.025263,-0.036316,-1.437500,0.294479,-0.033168,-0.018424,0.004506,0.004381,0.000111,-0.000084,0.000047,406.945179


In [3]:
# select the columns to be used for prediction
X_col = ['bull_D', 'bear_D', 'bull_W', 'bear_W', 'bull_M', 'bear_M', 'LMKT', 'IVOL', # initial columns
         'STR',	'LTURNOVER', 'IMOM', 'MOM',	'LTR', 'PCTHIGH', 'IVOL2', 'IVOL3', 'SPREAD_L',	# stock specific columns
         'MVOL', 'MVOL2', 'MVOL3', 'LMKT2',	'LMKT3', 'MMOM', 'MIMOM', 'MLTR', # market specific columns
         'LAT',	'DPI2A_L',	'CTO_L', 'D2A_L', 'NOA_L', 'OL_L',	'PCM_L', 'FC2Y_L',	
         'INVEST_L', 'RNA_L', 'S2E_L', 'PROF_L', 'PM_L', 'ATO_L', 'ROA_L', 	
         'FCF_L', 'A2ME_L',	'B2ME_L', 'S2P_L', 'ROE_L',	'LEV_L', 'TQ_L'] 

In [4]:
# scale the data for faster coeficient convergence
scaler = MinMaxScaler()

df[X_col] = scaler.fit_transform(df[X_col])

In [5]:
# Convert 'date' to datetime format (if not already done) and sort the DataFrame
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Create a 'year' column based on the 'date' column
df['year'] = df['date'].dt.year

# Paper Replication - OLS and Logit, Expanding Window - No Hyperparameters
- They start with out of sample forecasting in 1932
- models will be named model_default

### Linear Regression (Pooled OLS)

In [6]:
df.tail()

,PERMNO,date,y,RET,LME,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,SPREAD_L,LTURNOVER,IVOL,IVOL2,IVOL3,STR,LTR,IMOM,MOM,PCTHIGH,MVOL,MVOL2,MVOL3,LMKT,LMKT2,LMKT3,MMOM,MIMOM,MLTR,release_L,LAT,DPI2A_L,CTO_L,D2A_L,NOA_L,OL_L,PCM_L,FC2Y_L,INVEST_L,RNA_L,S2E_L,PROF_L,PM_L,ATO_L,ROA_L,FCF_L,A2ME_L,B2ME_L,S2P_L,ROE_L,LEV_L,TQ_L,year
1795158,14523,2022-07-31,0,-0.211268,4.210201e+05,3.603768e+05,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.010376,0.000127,0.000547,0.000081,0.000018,0.046569,0.752262,0.626562,0.667402,0.596191,0.108507,0.127252,0.072299,0.363567,0.568917,0.336051,0.428937,0.674246,0.832911,1.0,0.000078,0.007624,0.000401,0.000090,0.000694,0.000181,0.907457,0.105361,0.000694,0.261469,0.217470,0.304657,0.956337,0.008175,0.784626,0.140765,0.000122,0.797185,0.009079,0.522669,0.000005,0.000137,2022
1795159,84413,2022-07-31,1,0.020794,9.367426e+05,9.562860e+05,0.000000,0.0,0.000000,0.0,0.166667,0.0,0.004925,0.000068,0.000052,0.000111,0.000040,0.042502,0.708189,0.669056,0.712299,0.767522,0.108507,0.127252,0.072299,0.363567,0.568917,0.336051,0.428937,0.674246,0.832911,1.0,0.000262,0.006655,0.000400,0.000085,0.000376,0.000203,0.907473,0.105362,0.000376,0.261528,0.217475,0.304657,0.956344,0.008166,0.785350,0.140878,0.000094,0.797068,0.009113,0.522682,0.000030,0.000168,2022
1795160,14526,2022-07-31,1,0.066548,8.689975e+05,9.219528e+05,0.166667,0.0,0.083333,0.0,0.000000,0.0,0.004318,0.000094,0.000042,0.000013,0.000019,0.037465,0.704435,0.671810,0.725260,0.814332,0.108507,0.127252,0.072299,0.363567,0.568917,0.336051,0.428937,0.674246,0.832911,0.0,0.001048,0.006563,0.000466,0.000117,0.000394,0.000240,0.907461,0.105344,0.000394,0.261578,0.217481,0.304673,0.956347,0.008336,0.785690,0.141023,0.000383,0.797248,0.009861,0.522707,0.000150,0.000037,2022
1795161,90548,2022-07-31,1,0.087452,1.138132e+05,1.216028e+05,0.416667,0.0,0.083333,0.0,0.000000,0.0,0.005750,0.000025,0.000038,0.000066,0.000015,0.034937,0.710461,0.697203,0.745026,0.673998,0.108507,0.127252,0.072299,0.363567,0.568917,0.336051,0.428937,0.674246,0.832911,1.0,0.000372,0.006682,0.000408,0.000135,0.000382,0.000187,0.907466,0.105348,0.000382,0.261564,0.217474,0.304668,0.956347,0.008270,0.785608,0.141003,0.001059,0.797898,0.010453,0.522709,0.000321,0.000014,2022
1795162,93436,2022-07-31,1,0.323765,7.010302e+08,9.311106e+08,0.166667,0.0,0.083333,0.0,0.000000,0.0,0.004688,0.000218,0.000038,0.000041,0.000077,0.035278,0.866215,0.692366,0.729311,0.587806,0.108507,0.127252,0.072299,0.363567,0.568917,0.336051,0.428937,0.674246,0.832911,1.0,0.077968,0.006790,0.000415,0.000123,0.000400,0.000184,0.907462,0.105342,0.000400,0.261577,0.217473,0.304671,0.956348,0.008230,0.785736,0.141042,0.000032,0.797041,0.009040,0.522691,0.000006,0.000397,2022


# My Experiments

## Machine Learning - Hyperparameter Tuning included in the process
- models to be named 'model_clas/reg_exp/roll'

### First expanding, then rolling
start predicting for 1932, expand the window until you reach X years, then roll it

#### MSE Evaluation

##### 5 years

In [7]:
rolling_window = 5

In [8]:
############################################
# LASSO CLASSIFICATION MODEL - MSE
############################################

model_name = 'lasso_clas_roll5_MSE'  # Name of the new column for storing predictions
start_time2 = time.time()  # Start timing


# Predefined set of C values for hyperparameter tuning
HP1 = [0.01, 0.1, 1, 10, 100] # C

# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 6

for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
       
    best_HP1 = None
    best_mse = np.inf  # Initialize the best MSE to infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = LogisticRegression(C=hp1, max_iter=1000, penalty='l1', solver='saga')  # Ridge
        model.fit(X_train, y_train)
        probabilities = model.predict(X_tune)
        mse = mean_squared_error(y_tune, probabilities)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    
    # Retrain on the entire training window (excluding tuning year) with the best C value
    model = LogisticRegression(C=best_HP1, max_iter=1000, penalty='l1', solver='saga')
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_probabilities = model.predict_proba(X_next_year)[:, 1]  # Probability of the positive class
        df.loc[df['year'] == year, model_name] = next_year_probabilities
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best C: {best_HP1}, Best ACC: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")


end_time2 = time.time()  # End timing
print(f"Total time: {end_time2 - start_time2:.2f} seconds")

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 1976 - Best C: 100, Best ACC: 0.0812, Time: 288.11 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 1977 - Best C: 10, Best ACC: 0.0668, Time: 413.81 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 2021 - Best C: 10, Best ACC: 0.0335, Time: 629.65 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 2022 - Best C: 10, Best ACC: 0.045, Time: 613.99 seconds
Total time: 36842.77 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Forming Portfolios, Value-weighted portfolio returns

In [9]:
df.head()

,PERMNO,date,y,RET,LME,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,SPREAD_L,LTURNOVER,IVOL,IVOL2,IVOL3,STR,LTR,IMOM,MOM,PCTHIGH,MVOL,MVOL2,MVOL3,LMKT,LMKT2,LMKT3,MMOM,MIMOM,MLTR,release_L,LAT,DPI2A_L,CTO_L,D2A_L,NOA_L,OL_L,PCM_L,FC2Y_L,INVEST_L,RNA_L,S2E_L,PROF_L,PM_L,ATO_L,ROA_L,FCF_L,A2ME_L,B2ME_L,S2P_L,ROE_L,LEV_L,TQ_L,year,lasso_clas_roll5_MSE
0,31464,1970-04-30,0,-0.102941,41412.0,37149.0,0.0,0.000000,0.0,0.166667,0.0,0.416667,0.003421,0.000006,0.000046,0.000034,0.000047,0.039638,0.731272,0.612170,0.670081,0.413984,0.012852,0.012583,0.010420,0.564331,0.723312,0.385732,0.408401,0.496787,0.647683,1.0,0.000119,0.006682,0.000433,0.000149,0.000386,0.00062,0.907359,0.105362,0.000386,0.261588,0.217478,0.304686,0.956350,0.008189,0.785605,0.141009,0.001061,0.797682,0.010881,0.522705,0.000224,0.000016,1970,NaN
1,31464,1970-05-31,0,-0.065574,37149.0,34713.0,0.0,0.000000,0.0,0.000000,0.0,0.500000,0.004576,0.000006,0.000013,0.000046,0.000034,0.035635,0.722584,0.625955,0.672735,0.400651,0.017869,0.012852,0.012583,0.309299,0.564331,0.723312,0.385646,0.535558,0.630483,0.0,0.000119,0.006682,0.000433,0.000149,0.000386,0.00062,0.907359,0.105362,0.000386,0.261588,0.217478,0.304686,0.956350,0.008189,0.785605,0.141009,0.001061,0.797682,0.010881,0.522707,0.000250,0.000014,1970,NaN
2,31464,1970-06-30,0,-0.143860,34713.0,29232.0,0.0,0.000000,0.0,0.250000,0.0,0.583333,0.006530,0.000008,0.000062,0.000013,0.000046,0.037131,0.721443,0.622612,0.670797,0.455396,0.127382,0.017869,0.012852,0.414013,0.309299,0.564331,0.279599,0.494936,0.666119,0.0,0.000119,0.006682,0.000433,0.000149,0.000386,0.00062,0.907359,0.105362,0.000386,0.261588,0.217478,0.304686,0.956350,0.008189,0.785605,0.141009,0.001061,0.797682,0.010881,0.522708,0.000267,0.000013,1970,NaN
3,31464,1970-07-31,0,-0.125000,29232.0,25578.0,0.0,0.083333,0.0,0.000000,0.0,0.666667,0.004847,0.000004,0.000020,0.000062,0.000013,0.033998,0.698329,0.624735,0.678001,0.447986,0.035887,0.127382,0.017869,0.444076,0.414013,0.309299,0.282368,0.548597,0.589858,1.0,0.000119,0.006682,0.000424,0.000149,0.000386,0.00062,0.907347,0.105364,0.000386,0.261588,0.217477,0.304686,0.956351,0.008181,0.785544,0.140986,0.001061,0.797682,0.010711,0.522684,0.000316,0.000011,1970,NaN
4,31464,1970-08-31,1,0.047619,25578.0,26796.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.003877,0.000005,0.000047,0.000020,0.000062,0.034753,0.687449,0.638067,0.679698,0.391849,0.023573,0.035887,0.127382,0.766624,0.444076,0.414013,0.294738,0.536452,0.498298,0.0,0.000119,0.006682,0.000424,0.000149,0.000386,0.00062,0.907347,0.105364,0.000386,0.261588,0.217477,0.304686,0.956351,0.008181,0.785544,0.140986,0.001061,0.797682,0.010711,0.522684,0.000361,0.000010,1970,NaN


In [10]:
prediction_cols
# prediction_cols = ['logit_default','OLS_default','logit_roll6','DT_reg_roll']

['lasso_clas_roll5_MSE']

In [11]:
portfolio = df[['date', 'RET', 'ME', 'y'] + prediction_cols].copy()
portfolio['date'] = pd.to_datetime(portfolio['date'])

# drop rows with missing values
portfolio.dropna(inplace=True)

portfolio.head()

,date,RET,ME,y,lasso_clas_roll5_MSE
80751,1976-01-31,0.147541,99391.25,1,0.875884
80752,1976-01-31,0.266187,87802.00,1,0.979566
80753,1976-01-31,0.124031,113172.50,1,1.000000
80754,1976-01-31,0.319444,12801.25,1,1.000000
80755,1976-01-31,0.000000,8228.50,0,0.063539


In [12]:
portfolio.tail()

,date,RET,ME,y,lasso_clas_roll5_MSE
1795158,2022-07-31,-0.211268,3.603768e+05,0,0.003037
1795159,2022-07-31,0.020794,9.562860e+05,1,1.000000
1795160,2022-07-31,0.066548,9.219528e+05,1,0.982230
1795161,2022-07-31,0.087452,1.216028e+05,1,0.999221
1795162,2022-07-31,0.323765,9.311106e+08,1,0.998633


In [13]:
# Initialize an empty DataFrame to store value-weighted returns for each model
vwreturns = pd.DataFrame(portfolio['date'].unique(), columns=['date'])  # Ensures all dates are included

for pred_col in prediction_cols:
    # Calculate deciles for this prediction
    decile_col = f'decile_{pred_col}'
    portfolio[decile_col] = portfolio.groupby(['date'])[pred_col].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))
    
    # Determine position based on deciles
    position_col = f'position_{pred_col}'
    portfolio[position_col] = np.where(portfolio[decile_col] == 9, 1, np.where(portfolio[decile_col] == 0, -1, 0))
    
    # Calculate the value-weighted return for this prediction
    vwret_col = f'vwreturn_{pred_col}'
    vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)
    
    # Merge the temporary value-weighted returns with the main vwreturns DataFrame
    vwreturns = vwreturns.merge(vwreturns_temp, on='date', how='left')

# Ensure the 'date' column is the first column and is sorted
vwreturns = vwreturns.sort_values('date').reset_index(drop=True)


/tmp/ipykernel_18/2999037948.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)


In [14]:
vwreturns.head()

,date,vwreturn_lasso_clas_roll5_MSE
0,1976-01-31,-0.001181
1,1976-02-29,0.003720
2,1976-03-31,0.002116
3,1976-04-30,0.002856
4,1976-05-31,0.016825


### Compare to market data

In [15]:
#market = pd.read_csv('FF3_clean.csv')
market = pd.read_csv('/kaggle/input/sign-prediction-datasets/FF3_clean.csv')

In [16]:
market.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-31,2.96,-2.56,-2.43,0.22
1,1926-08-31,2.64,-1.17,3.82,0.25
2,1926-09-30,0.36,-1.40,0.13,0.23
3,1926-10-31,-3.24,-0.09,0.70,0.32
4,1926-11-30,2.53,-0.10,-0.51,0.31


In [17]:
# create a new 'Mkt' which is a sum of Mkt-RF and RF
market['Mkt'] = market['Mkt-RF'] + market['RF']

# divide all columns by 100 except 'date'
market.iloc[:, 1:] = market.iloc[:, 1:] / 100

#set the 'date' column to datetime format
market['date'] = pd.to_datetime(market['date'])

# merge the market data (only date and Mkt columns) with the vwreturns DataFrame
vwreturns = vwreturns.merge(market[['date', 'Mkt']], on='date', how='left')

# transform all columns (except 'date') to a log: log(x+1) and save the result as lvwreturns
lvwreturns = vwreturns.copy()
lvwreturns.iloc[:, 1:] = np.log(vwreturns.iloc[:, 1:] + 1)

In [18]:
vwreturns.head()

,date,vwreturn_lasso_clas_roll5_MSE,Mkt
0,1976-01-31,-0.001181,0.1263
1,1976-02-29,0.003720,0.0066
2,1976-03-31,0.002116,0.0272
3,1976-04-30,0.002856,-0.0107
4,1976-05-31,0.016825,-0.0097


In [19]:
lvwreturns.head()

,date,vwreturn_lasso_clas_roll5_MSE,Mkt
0,1976-01-31,-0.001181,0.118938
1,1976-02-29,0.003713,0.006578
2,1976-03-31,0.002113,0.026837
3,1976-04-30,0.002852,-0.010758
4,1976-05-31,0.016685,-0.009747


In [20]:
lvwreturns.describe()

,date,vwreturn_lasso_clas_roll5_MSE,Mkt
count,559,559.000000,559.000000
mean,1999-05-01 04:22:45.295169920,0.005775,0.009303
min,1976-01-31 00:00:00,-0.024715,-0.256700
25%,1987-09-15 00:00:00,0.002053,-0.015672
50%,1999-04-30 00:00:00,0.003340,0.013804
75%,2010-12-15 12:00:00,0.008377,0.038162
max,2022-07-31 00:00:00,0.037848,0.127953
std,NaN,0.005694,0.045169


In [21]:
# save the lvwreturns and portfolio DataFrame to a parquet file into 'outputs' folder

# for reproducibility and visualization purposes
lvwreturns.to_parquet('fin_lvwreturns_class2.parquet')
portfolio.to_parquet('fin_portfolio_class2.parquet')

# save vwreturns DataFrame to a .dta file into 'outputs' folder
#vwreturns.to_stata('outputs/vwreturns.dta') # for backtasting in R - we need normal returns, not log returns
